# Intralayer interatomic distance analysis of LAMMPS MD trajectory

## Header

In [ ]:
import lmpdump as lmpdump    # Use lmpdump.py parser
from sitator.util import PBCCalculator    # Use PBC calculator from sitator package
import os
import math
import numpy as np
import time
from tqdm import tqdm

pwd = os.getcwd()

print('Please cite DOI: 10.1021/acs.jpcc.9b04863.')
print('This script analyzes intralayer interatomic distances from LAMMPS MD trajectory.')
print('This script is interactive and requires user input. Please read carefully before proceeding:\n')

print('Note 1: Intended only for periodic slab models of up to bimetallic FCC systems, with vacuum along the z-direction.')
print('Note 2: Intended only for LAMMPS NVT simulations.\n')

print('Note 3: Requires a LAMMPS DATA file containing the equilibrated box dimensions.')
print('Note 4: Requires a trajectory ordered by atom ID, unscaled, wrapped.')
print('Note 5: Requires lmpdump.py to load the LAMMPS trajectories.')
print('Note 6: Requires PBC calculator from sitator package.\n')

## Load LAMMPS trajectory

In [ ]:
# LAMMPS custom-style dump file - Clamped trajectory
name = input('Enter the name of the LAMMPS trajectory file (ID, type, x, y, z) : \nWarning: Must be ordered by atom ID, unscaled, and wrapped! Please quit now if not so.\n')
print('Loading trajectory...')
file = pwd + '/' + name
xyz = lmpdump.lmpdump(file, loadmode='all')
print('\n')

print('Loading time steps...')
step = []
for s in xyz.finaldict.keys():    # keys = time steps
    step.append(s)
print('Done.\n')

size = np.array(step).size    # Number of time steps
N_dump = xyz.finaldict[step[0]][1].shape[0]    # Number of atoms dumped
xlo_ext = xyz.finaldict[step[0]][0][0]    # x_ext = x + xy
xhi_ext = xyz.finaldict[step[0]][0][1]

## Extract unit cell information

In [ ]:
# Load LAMMPS data file after NPT equilibration
name = input('Enter the name of the LAMMPS DATA file containing the equilibrated box dimensions : ')
file_eq = pwd + '/' + name
log = open(file_eq,'r')
log_lines = log.readlines()
log_lines = [line.split() for line in log_lines]

for line_index, line in enumerate(log_lines):
    if line:
        for l in line:
            
            if l == 'atoms':
                N_atom = int(line[0])
                
            elif l == 'types':
                N_type = int(line[0])

            elif l == 'xlo':
                xlo = float(line[0])
                xhi = float(line[1])
            
            elif l == 'ylo':
                ylo = float(line[0])
                yhi = float(line[1])
            
            elif l == 'zlo':
                zlo = float(line[0])
                zhi = float(line[1])
            
            elif l == 'xy':
                xy = float(line[0])
                xz = float(line[1])
                yz = float(line[2])
            
            elif l == 'Atoms':
                head = line_index + 1
                start = line_index + 2
            
            elif l == 'Velocities':
                end = line_index - 1

Lx = xhi - xlo    # Box dimensions
Ly = yhi - ylo
Lz = zhi - zlo
tan = Ly/xy

lat = np.array([[xhi-xlo, 0.0, 0.0], [xy, yhi-ylo, 0.0], [xz, yz, zhi-zlo]])    # Lattice matrix
pbc = PBCCalculator(lat)    # Periodic boundary condition

ID2 = int(input('Enter the ID of a nearest-neighbor atom that lies in the same layer as atom #1 : '))
ID3 = int(input('Enter the ID of another nearest-neighbor atom that lies in the same layer as atom #1 : '))
ID4 = int(input('Enter the ID of an atom that lies one layer above atom #1 : '))

for line in log_lines[start:end]:

    line[:2] = np.array(line[:2]).astype(int)    # Atom ID, atom type
    line[2:5] = np.array(line[2:5]).astype(float)    # x, y, z
    line[5:8] = np.array(line[5:8]).astype(int)    # ix, iy, iz (inverse sign)
            
    x = line[2] + line[5]*Lx + line[6]*xy    # Wrapped coordinates
    y = line[3] + line[6]*Ly
    z = line[4] + line[7]*Lz
    
    if line[0] == 1:
        atom1 = np.array([x,y,z])
    
    elif line[0] == ID2:
        atom2 = np.array([x,y,z])
    
    elif line[0] == ID3:
        atom3 = np.array([x,y,z])
    
    elif line[0] == ID4:
        atom4 = np.array([x,y,z])

r12 = np.linalg.norm(np.subtract(atom1,atom2))
r13 = np.linalg.norm(np.subtract(atom1,atom3))
dr_avg = np.mean([r12, r13])    # In-plane NN distance

a1 = np.subtract(atom2,atom1)    # 1st intralayer vector
a1 = a1 / np.linalg.norm(a1)

a2 = np.subtract(atom3,atom1)    # 2nd intralayer vector
a2 = a2 / np.linalg.norm(a2)

a3 = np.cross(a1,a2)    # Normal vector
if a3[2] < 0:
    a3 = -a3    # Ensure normal vector along +z
a3 = a3 / np.linalg.norm(a3)

dn_avg = np.dot(atom4,a3) - np.dot(atom1,a3)    # Interlayer distance; n = normal component

N_slab = int(input('Enter the number of layers in the slab, excluding the deposit/adsorbate layer(s) : '))
if xyz.finaldict[step[0]][1]['id'][0] != 1:    # Is bottommost layer muted?
    N_slab -= 1
    
z_atom = []    # Atomic height
for atom in range(0,N_dump):
    xs = xyz.finaldict[step[-1]][1]['xs'][atom]    # Use last frame
    ys = xyz.finaldict[step[-1]][1]['ys'][atom]
    zs = xyz.finaldict[step[-1]][1]['zs'][atom]
    rs = np.array([xs,ys,zs])
    r = np.dot(lat,rs)
    z_atom.append(r[2])
z_atom = np.array(z_atom)
z_atom = np.sort(z_atom)    # Sort by ascending height

atom_next = 0
z_layer = []    # Average layer heights
while atom_next < N_dump-1:
    z_bin = []
    
    for atom in range(atom_next,N_dump):
        z0 = z_atom[atom]    # Use last frame
        z1 = z_atom[atom+1]
        dz = z1 - z0
        
        if abs(dz) > 0.20*dn_avg:
            atom_next = atom+1
            break
            
        elif z0 not in z_bin:
            z_bin.append(z0)
        
        if atom == N_dump-2:
            atom_next = atom+1
            break
        
    z_bin = np.array(z_bin)
    z_avg = np.mean(z_bin)
    z_layer.append(z_avg)
    
N_layer = len(z_layer)

log.close()

print('Unit cell information & normal direction extracted.\n')

## Intralayer interatomic distance analysis

In [ ]:
file_ravg = pwd + '/ravg.txt'
out_ravg = open(file_ravg,'w')

header = 'time (ns)\t'
for l in range(0,N_layer):
    name = 'L' + str(l)
    header += name + '\t'
header += '\n'
out_ravg.write(header)

dt = int(input('Enter the simulation timestep in fs : '))

for s_index, s in enumerate(tqdm(step)):
    t = s*dt/(10**6)
    out = str(t) + '\t'
    
    layer = {}    # Sort atoms by layer
    for l in range(0,N_layer):
        name = 'L' + str(l)
        layer[name] = []
    
    for atom in range(0,N_dump):
        
        xs = xyz.finaldict[s][1]['xs'][atom]
        ys = xyz.finaldict[s][1]['ys'][atom]
        zs = xyz.finaldict[s][1]['zs'][atom]
        rs = np.array([xs,ys,zs])
        
        r = np.dot(lat,rs)
        z = r[2]
        
        for l in range(0,N_layer):
            name = 'L' + str(l)
            z_ref = z_layer[l]
            dz = z - z_ref
            if abs(dz) < 0.20*dn_avg:
                layer[name].append(r)
                break
    
    for l in range(0,N_layer):
        name = 'L' + str(l)
        layer[name] = np.array(layer[name])
        
        N = layer[name].shape[0]
        if N <= 1:
            rij_avg = 0.0
            out += str(rij_avg) + '\t'
            continue
        
        rij = pbc.pairwise_distances(layer[name])    # Pairwise distance matrix        
        rij_bin = []
        for i in range(0,N):
            for j in range(0,N):
                if j > i:
                    if rij[i][j] < 1.20*dr_avg:
                        rij_bin.append(rij[i][j])                        
        rij_bin = np.array(rij_bin)

        if rij_bin.shape[0] == 0:
            rij_avg = 0.0
        else:
            rij_avg = np.mean(rij_bin)
        
        out += str(rij_avg) + '\t'
    
    out += '\n'
    out_ravg.write(out)
    
out_ravg.close()